<a href="https://colab.research.google.com/github/tambeyash/kaggle_galaxy_zoo/blob/master/galaxy_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math 
import datetime
import time

In [0]:
#importing the classes
classes = pd.read_csv('training_solutions_rev1.csv')

In [20]:
classes.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [21]:
classes.shape

(61578, 38)

In [22]:
c1 = classes.T
c1.head()

,0,1,2,3,4,5,6,7,8,9,...,61568,61569,61570,61571,61572,61573,61574,61575,61576,61577
GalaxyID,100008.000000,100023.000000,100053.000000,100078.000000,100090.000000,100122.000000,100123.000000,100128.000000,100134.000000,100143.000000,...,999875.000000,999890.000000,999893.000000,999900.000000,999936.000000,999948.000000,999950.000000,999958.000000,999964.00000,999967.000
Class1.1,0.383147,0.327001,0.765717,0.693377,0.933839,0.738832,0.462492,0.687783,0.021834,0.269843,...,0.020285,0.308029,0.410999,0.460239,0.545443,0.510379,0.901216,0.202841,0.09100,0.767
Class1.2,0.616853,0.663777,0.177352,0.238564,0.000000,0.238159,0.456033,0.288344,0.976952,0.730157,...,0.979715,0.691971,0.519560,0.511396,0.454557,0.489621,0.098784,0.777376,0.90900,0.140
Class1.3,0.000000,0.009222,0.056931,0.068059,0.066161,0.023009,0.081475,0.023873,0.001214,0.000000,...,0.000000,0.000000,0.069441,0.028365,0.000000,0.000000,0.000000,0.019783,0.00000,0.093
Class2.1,0.000000,0.031178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021751,0.730157,...,0.207551,0.691971,0.051956,0.109439,0.056820,0.059207,0.000000,0.116962,0.04545,0.000


In [23]:
c2 = c1.drop('GalaxyID', axis = 0)
c2.head()

,0,1,2,3,4,5,6,7,8,9,...,61568,61569,61570,61571,61572,61573,61574,61575,61576,61577
Class1.1,0.383147,0.327001,0.765717,0.693377,0.933839,0.738832,0.462492,0.687783,0.021834,0.269843,...,0.020285,0.308029,0.410999,0.460239,0.545443,0.510379,0.901216,0.202841,0.09100,0.767
Class1.2,0.616853,0.663777,0.177352,0.238564,0.000000,0.238159,0.456033,0.288344,0.976952,0.730157,...,0.979715,0.691971,0.519560,0.511396,0.454557,0.489621,0.098784,0.777376,0.90900,0.140
Class1.3,0.000000,0.009222,0.056931,0.068059,0.066161,0.023009,0.081475,0.023873,0.001214,0.000000,...,0.000000,0.000000,0.069441,0.028365,0.000000,0.000000,0.000000,0.019783,0.00000,0.093
Class2.1,0.000000,0.031178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021751,0.730157,...,0.207551,0.691971,0.051956,0.109439,0.056820,0.059207,0.000000,0.116962,0.04545,0.000
Class2.2,0.616853,0.632599,0.177352,0.238564,0.000000,0.238159,0.456033,0.288344,0.955201,0.000000,...,0.772164,0.000000,0.467604,0.401957,0.397737,0.430414,0.098784,0.660414,0.86355,0.140


In [0]:
y = c2.idxmax(axis=0)

In [0]:
y1 = np.asarray(y)

In [26]:
y1

array(['Class6.2', 'Class6.2', 'Class6.2', ..., 'Class6.1', 'Class1.2',
       'Class1.1'], dtype=object)

In [27]:
np.unique(y1)

array(['Class1.1', 'Class1.2', 'Class1.3', 'Class6.1', 'Class6.2'],
      dtype=object)

In [12]:
#freq_count(y1)

NameError: ignored

In [16]:
#d = dict([(i,0) for i in np.unique(y1)])
#for number in y1:
#    d[j]+=1

NameError: ignored

In [0]:
!unzip file_location

In [0]:
#This is the best model we found. For additional models, check out I_notebook.ipynb
start = datetime.datetime.now()
model = Sequential() 
model.add(Flatten(input_shape=train_data.shape[1:])) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.5)) 
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation=’softmax’))
model.compile(loss=’categorical_crossentropy’,
   optimizer=optimizers.RMSprop(lr=1e-4),
   metrics=[‘acc’])
history = model.fit(train_data, train_labels, 
   epochs=7,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print(“[INFO] accuracy: {:.2f}%”.format(eval_accuracy * 100)) 
print(“[INFO] Loss: {}”.format(eval_loss)) 
end= datetime.datetime.now()
elapsed= end-start
print (‘Time: ‘, elapsed)